### Prologue
An attempt at implementing [Denoising Induction Motor Sounds](https://arxiv.org/pdf/2208.04462.pdf) for this use case.

### IMPORTS

In [2]:
from __future__ import print_function, division
%matplotlib inline

import sys, time, random, glob, os, pandas
import numpy as np
import librosa
import pydub
import json
from pydub import AudioSegment
from pydub import effects

import tensorflow as tf
import keras

from keras.utils import np_utils
from keras import optimizers
from keras.layers import Convolution2D, MaxPooling2D

from tensorflow.keras import layers, losses

import keras.models
from keras.losses import mse as kmse
import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Add, Multiply, Lambda, UpSampling2D, Dot, Permute, RepeatVector
from keras.layers import BatchNormalization
from keras.layers import Conv2D
from keras.layers import Cropping2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Input
from keras.models import Model

from tqdm.notebook import tqdm

from keras.callbacks import LambdaCallback, EarlyStopping, ModelCheckpoint
import noising
from common import *
from common import cal_midpoints, gen_mel_feature

In [ ]:
INPUT_DIR = 'samples'
files = glob.glob(f'{INPUT_DIR}/*.flac')
noising.save_overlaid_dataset(files, 3, 15, 'dataset.json', 'overlaid', 150)

In [6]:
class FastGenerator(tf.keras.utils.Sequence):
    def __init__(self, directory : str = 'samples', 
                train_ratio : float = 0.8, window_size : int = 49000,
                batch_size : int = 128, gen_type = 'train', shuffle : bool = True):
        self.shuffle = shuffle
        self.gen_type = gen_type
        self.train_ratio = train_ratio
        self.test_ratio = self.train_ratio + (1-train_ratio)/2
        self.batch_size = batch_size
        self.epoch = 0
        self.window_size = window_size
        self.noise_types = ['white', 'pink', 'blue', 'brown', 'violet']
        files = glob.glob(f'{directory}/*.flac')
        idx = int(train_ratio*len(files))
        test_idx = int(self.test_ratio*len(files))
        if gen_type == 'train':
            self.files = files[0:idx]
        elif gen_type == 'test':
            self.files = files[idx:test_idx]
        else:
            self.files = files[test_idx:]
        print(f"{gen_type} loader created with {len(self.files)} samples")
        
        self.len = self.__len__()
        if self.gen_type != 'test':
            self.on_epoch_end()

    def __len__(self):
        '''Denotes the number of batches per epoch'''
        return (2*len(self.files)) // self.batch_size

    def on_epoch_end(self):
        if self.shuffle and self.gen_type != 'test' :
            random.shuffle(self.files)
        self.epoch += 1

    def __getitem__(self, index):
        #! generates self.samples_per_file noisy samples per file, can be of the
        #! same kind of noise, or can be of different kinds of noises.
        inp = []
        out = []
        const = 1 #! const files make up a batch in case of 128
        for path in self.files[index*const : (index+1)*const]:
            clear, noisy = noising.chop_overlay_spit(path, self.window_size, self.noise_types)
            inp += noisy
            out += clear
        inp = np.array(inp)
        out = np.array(out)
        return inp, out

In [8]:
class Generator(tf.keras.utils.Sequence):
    def __init__(self, directory : str = '/home/juggernautjha/Desktop/to_rahul/maruti/data/true_samples', 
                length : int = 200, train_ratio : float = 0.8,
                batch_size : int = 32, gen_type = 'train',
                samples_per_file : int = 3, shuffle : bool = True):
        self.shuffle = shuffle
        self.gen_type = gen_type
        self.train_ratio = train_ratio
        self.test_ratio = self.train_ratio + (1-train_ratio)/2
        self.batch_size = batch_size
        self.epoch = 0
        self.pad_to = length
        self.samples_per_file = samples_per_file
        self.noise_types = ['white', 'pink', 'blue', 'brown', 'violet']
        files = glob.glob(f"{directory}/*.flac")
        idx = int(train_ratio*len(files))
        test_idx = int(self.test_ratio*len(files))
        if gen_type == 'train':
            self.files = files[0:idx]
        elif gen_type == 'test':
            self.files = files[idx:test_idx]
        else:
            self.files = files[test_idx:]
        print(f"{gen_type} loader created with {len(self.files)} samples")

        self.len = self.__len__()
        if self.gen_type != 'test':
            self.on_epoch_end()

    def __len__(self):
        '''Denotes the number of batches per epoch'''
        return (2*len(self.files)) // self.batch_size

    def on_epoch_end(self):
        if self.shuffle and self.gen_type != 'test' :
            random.shuffle(self.files)
        self.epoch += 1

    def __getitem__(self, index):
        #! generates self.samples_per_file noisy samples per file, can be of the
        #! same kind of noise, or can be of different kinds of noises.
        inp = []
        out = []
        const = int(self.batch_size//self.samples_per_file) #! const files make up a batch
        for path in self.files[index*const : (index+1)*const]:
            colors = random.choices(self.noise_types, k =self.samples_per_file)
            for color in colors:
                noisy, clear = noising.overlay_noise(path, 15, color, self.pad_to)
                inp.append(noisy.get_array_of_samples())
                out.append(clear.get_array_of_samples())
        inp = np.array(inp)
        out = np.array(out)
        return inp, out
    

In [7]:
train_generator = FastGenerator()
val_generator = FastGenerator(gen_type='val')

train loader created with 16 samples
val loader created with 2 samples


In [8]:
len(train_generator.__getitem__(1)[0][1])

  0%|          | 0/128 [00:00<?, ?it/s]

49000

##### MODEL (finally??)

In [9]:
class Denoise(Model):
  def __init__(self, input_size, batch_size):
    super(Denoise, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Input(shape = (input_size, 1), batch_size = batch_size),
      layers.Conv1D(128, kernel_size=3, activation='relu', kernel_initializer='he_uniform'),
      layers.Conv1D(32, kernel_size=3, activation='relu', kernel_initializer='he_uniform'),
      layers.Conv1D(16, kernel_size=3, activation='relu', kernel_initializer='he_uniform'),
      layers.Conv1D(8, kernel_size=3, activation='relu', kernel_initializer='he_uniform')
      ])

    self.decoder = tf.keras.Sequential([
      layers.Conv1DTranspose(8, kernel_size=3, activation='relu', kernel_initializer='he_uniform'),
      layers.Conv1DTranspose(16, kernel_size=3, activation='relu', kernel_initializer='he_uniform'),
      layers.Conv1DTranspose(32, kernel_size=3, activation='relu', kernel_initializer='he_uniform'),
      layers.Conv1DTranspose(128, kernel_size=3, activation='relu', kernel_initializer='he_uniform'),
      layers.Conv1DTranspose(1, kernel_size=3, activation='relu', kernel_initializer='he_uniform')
      ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

In [10]:
denoiser = Denoise(49000, 128)

2023-06-24 01:50:43.250927: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-24 01:50:43.350105: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [11]:
denoiser.compile('adam', loss=tf.keras.losses.mean_squared_error)

In [12]:
denoiser.fit(train_generator, validation_data=val_generator)

  0%|          | 0/128 [00:00<?, ?it/s]

2023-06-24 01:51:22.017169: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25088000 exceeds 10% of free system memory.
2023-06-24 01:51:22.036622: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 50176000 exceeds 10% of free system memory.
2023-06-24 01:51:22.083972: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3211132928 exceeds 10% of free system memory.
